In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_cpu_320x320_coco_2020_05_19.tar.gz
!tar -xzvf "/content/ssdlite_mobiledet_cpu_320x320_coco_2020_05_19.tar.gz" -C "/content/"

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_mnasfpn_shared_box_predictor_320x320_coco_sync_2020_05_18.tar.gz
!tar -xzvf "/content/ssd_mobilenet_v2_mnasfpn_shared_box_predictor_320x320_coco_sync_2020_05_18.tar.gz" -C "/content/"

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz
!tar -xzvf "/content/ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz" -C "/content/"

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz
!tar -xzvf "/content/ssd_mobilenet_v3_small_coco_2020_01_14.tar.gz" -C "/content/"

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!tar -xzvf "/content/ssd_mobilenet_v2_coco_2018_03_29.tar.gz" -C "/content/"

In [ ]:
!git clone https://github.com/tensorflow/tensorflow.git
#!git clone https://github.com/tensorflow/models.git
#!git clone --branch archive https://github.com/tensorflow/models.git

In [ ]:
# Download Latest version of Bazelisk
!wget https://github.com/bazelbuild/bazelisk/releases/latest/download/bazelisk-linux-amd64

# Make script executable
!chmod +x bazelisk-linux-amd64

# Adding to the path
!sudo mv bazelisk-linux-amd64 /usr/local/bin/bazel


In [ ]:
# Bazel info.
!bazel

In [ ]:
cd tensorflow

In [ ]:
!ls

In [ ]:
!bazel run -c opt tensorflow/lite/python:tflite_convert -- \
--enable_v1_converter \
--graph_def_file='/content/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb' \
--output_file='/content/ssd_mobilenet_v2_coco_2018_03_29/detect.tflite' \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=QUANTIZED_UINT8 \
--mean_values=128 \
--std_dev_values=128 \
--change_concat_input_ranges=false \
--allow_custom_ops

In [ ]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
print(input_details)
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array([[0.195312]], dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

In [ ]:
cd ../

In [ ]:
%env CONFIG_FILE='/content/ssd_mobilenet_v2_coco_2018_03_29/pipeline.config'
%env CHECKPOINT_PATH='/content/ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.meta'
%env OUTPUT_DIR=/tmp/tflite

!export -p

In [ ]:
!find . -name "setup.py"

In [ ]:
#!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
!sudo python setup.py install

In [ ]:
!pip install tf_slim

In [ ]:
#!protoc -I=./ --python_out=./ ./object_detection/protos/*.proto
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!python object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=$CONFIG_FILE \
--trained_checkpoint_prefix=$CHECKPOINT_PATH \
--output_directory=$OUTPUT_DIR \
--add_postprocessing_op=true